# Imports + Drive + paths

In [1]:
!pip install mediapipe==0.10.14 opencv-python==4.10.0.84 scikit-learn tensorflowjs --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 43.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires packaging>=21, but you have packaging 20.9 which is incompatible.
db-dtypes 1.4.4 requires packaging>=24.2.0, but you have packaging 20.9 which is incompatible.
opentelemetry-proto 1.37.0 requires pro

In [22]:
# 0. IMPORTS BÁSICOS
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import json
import random
import numpy as np
from collections import defaultdict

import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 🔒 Semilla global para que el entrenamiento sea estable
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TF version:", tf.__version__)

# Rutas base en tu Drive
BASE_DIR = "/content/drive/MyDrive/SignProject"
RAW_DIR = os.path.join(BASE_DIR, "raw_videos")
PROCESSED_DIR = os.path.join(BASE_DIR, "processed")
MODELS_DIR = os.path.join(BASE_DIR, "models")

os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

print("RAW_DIR:", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("MODELS_DIR:", MODELS_DIR)

# Constantes del modelo
MAX_FRAMES = 20      # puedes subir a 40 si tus gestos son largos
N_FEATURES = 225     # 75 puntos * (x,y,z)

mp_holistic = mp.solutions.holistic


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TF version: 2.19.0
RAW_DIR: /content/drive/MyDrive/SignProject/raw_videos
PROCESSED_DIR: /content/drive/MyDrive/SignProject/processed
MODELS_DIR: /content/drive/MyDrive/SignProject/models


1. Función de extracción de landmarks (misma lógica que entrenamiento)

In [25]:
def extract_landmarks_from_results(results):
    """
    Convierte los resultados de MediaPipe Holistic en un vector 1D (225,)
    con pose (33), mano izq (21) y mano der (21).
    Cada punto = (x, y, z) => 75 * 3 = 225 features.
    """
    def get_xyz(landmarks, n_points):
        if landmarks is None:
            data = [[0.0, 0.0, 0.0]] * n_points
        else:
            data = [[lm.x, lm.y, lm.z] for lm in landmarks]
            if len(data) < n_points:
                # Rellenar si faltan
                data += [[0.0, 0.0, 0.0]] * (n_points - len(data))
            data = data[:n_points]
        return data

    pose = get_xyz(results.pose_landmarks.landmark if results.pose_landmarks else None, 33)
    left_hand = get_xyz(results.left_hand_landmarks.landmark if results.left_hand_landmarks else None, 21)
    right_hand = get_xyz(results.right_hand_landmarks.landmark if results.right_hand_landmarks else None, 21)

    all_points = pose + left_hand + right_hand
    return np.array(all_points, dtype=np.float32).flatten()  # (225,)


def mirror_sequence(seq):
    """
    Recibe una secuencia (T, 225) y devuelve una versión espejada:
      - Refleja en horizontal: x -> 1.0 - x
      - Intercambia mano izquierda y mano derecha

    Asume el orden:
      0-32  -> pose (33 puntos)
      33-53 -> mano izquierda (21 puntos)
      54-74 -> mano derecha (21 puntos)
    """
    if seq.ndim != 2 or seq.shape[1] != 225:
        raise ValueError(f"mirror_sequence espera (T, 225), recibió {seq.shape}")

    T, D = seq.shape
    pts = seq.reshape(T, 75, 3).copy()  # (T, 75, 3)

    # 1) Espejar en eje X (horizontal)
    pts[..., 0] = 1.0 - pts[..., 0]

    # 2) Separar segmentos: pose, mano izq, mano der
    pose   = pts[:, :33, :]      # (T, 33, 3)
    left   = pts[:, 33:54, :]    # (T, 21, 3)
    right  = pts[:, 54:75, :]    # (T, 21, 3)

    # 3) Reconstruir como: pose + (mano derecha como izquierda) + (mano izquierda como derecha)
    pts_mirrored = np.concatenate([pose, right, left], axis=1)  # (T, 75, 3)

    # 4) Volver a (T, 225)
    return pts_mirrored.reshape(T, 225).astype(np.float32)



**2. Construcción del dataset secuencial para LSTM**

Cada video = 1 muestra
Cada muestra = secuencia de max_frames frames, cada frame = 225 features.

In [43]:
def pad_or_truncate(seq, max_frames=MAX_FRAMES):
    """
    Asegura que cada secuencia tenga exactamente max_frames frames.
    - Si hay más frames, recorta centrado.
    - Si hay menos, rellena con ceros al final.
    seq: (T, 225)
    """
    n = seq.shape[0]

    if n == max_frames:
        return seq.astype(np.float32)

    if n > max_frames:
        start = max(0, (n - max_frames) // 2)
        return seq[start:start + max_frames].astype(np.float32)

    # n < max_frames → padding con ceros al final
    pad_len = max_frames - n
    pad = np.zeros((pad_len, seq.shape[1]), dtype=np.float32)
    return np.concatenate([seq, pad], axis=0).astype(np.float32)


def random_temporal_crop(seq, max_frames=MAX_FRAMES, jitter=10):
    """
    Data augmentation temporal:
    - Si el video es más largo que max_frames, recortamos una ventana
      de max_frames pero moviendo el inicio (para que el gesto pueda
      aparecer en distintas partes del clip).
    - Si es más corto, delegamos a pad_or_truncate.
    """
    n = seq.shape[0]

    if n <= max_frames:
        return pad_or_truncate(seq, max_frames=max_frames)

    max_start = n - max_frames
    center = max_start // 2
    start_min = max(0, center - jitter)
    start_max = min(max_start, center + jitter)

    start = np.random.randint(start_min, start_max + 1)
    return seq[start:start + max_frames].astype(np.float32)


def build_sequence_dataset_from_raw(
    RAW_DIR,
    outfile="landmarks_sequences_v1.npz",
    max_frames=MAX_FRAMES,
    samples_per_video=5,
    jitter=10
):
    """
    Crea un dataset secuencial:
      X_seq -> (n_samples, max_frames, 225)
      y_seq -> (n_samples,)
    Cada video original genera 'samples_per_video' secuencias con
    recortes temporales distintos, para que el gesto pueda aparecer
    en diferentes posiciones dentro de la ventana.
    """
    classes = sorted(os.listdir(RAW_DIR))
    classes = [c for c in classes if os.path.isdir(os.path.join(RAW_DIR, c))]
    print("Clases encontradas:", classes)

    label_names = classes
    X_seq_list = []
    y_seq_list = []

    for label_idx, cls_name in enumerate(label_names):
        class_dir = os.path.join(RAW_DIR, cls_name)
        video_files = [
            f for f in os.listdir(class_dir)
            if f.lower().endswith((".mp4", ".mov", ".avi", ".mkv"))
        ]
        print(f"\nClase '{cls_name}': {len(video_files)} videos")

        for fname in video_files:
            video_path = os.path.join(class_dir, fname)
            print("  Procesando:", video_path)

            cap = cv2.VideoCapture(video_path)
            frames_feats = []

            with mp_holistic.Holistic(
                static_image_mode=False,
                model_complexity=1,
                enable_segmentation=False,
                refine_face_landmarks=False,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5,
            ) as holis:

                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = holis.process(frame_rgb)

                    feats = extract_landmarks_from_results(results)  # (225,)
                    frames_feats.append(feats)

            cap.release()

            if len(frames_feats) == 0:
                print("    ⚠️  Video sin landmarks, se omite.")
                continue

            seq_full = np.stack(frames_feats, axis=0)  # (T, 225)

            # 🔹 NUEVO: secuencia original + secuencia espejada
            base_sequences = [seq_full, mirror_sequence(seq_full)]

            # Data augmentation temporal: varias secuencias por video
            for base in base_sequences:
                for _ in range(samples_per_video):
                    seq_sample = random_temporal_crop(
                        base,
                        max_frames=max_frames,
                        jitter=jitter
                    )
                    X_seq_list.append(seq_sample)
                    y_seq_list.append(label_idx)

    X_seq = np.stack(X_seq_list, axis=0).astype(np.float32)
    y_seq = np.array(y_seq_list, dtype=np.int64)
    label_names = np.array(label_names, dtype=object)

    print("\nShape final X_seq:", X_seq.shape)
    print("Shape final y_seq:", y_seq.shape)

    out_path = os.path.join(PROCESSED_DIR, outfile)
    np.savez(out_path, X_seq=X_seq, y_seq=y_seq, label_names=label_names)
    print("✅ Dataset guardado en:", out_path)

    return X_seq, y_seq, label_names


# EJECUTAR ESTA CELDA PARA CREAR / ACTUALIZAR EL DATASET
X_seq, y_seq, label_names = build_sequence_dataset_from_raw(
    RAW_DIR,
    outfile="landmarks_sequences_v2.npz",  # version 2 nuevo nombre
    max_frames=MAX_FRAMES,
    samples_per_video=3,
    jitter=12  # un poquito mayor
)



Clases encontradas: ['Disculpa', 'Inyeccion ', 'Marron', 'Por favor', 'Resfriado', 'Tos', 'Verde']

Clase 'Disculpa': 3 videos
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Disculpa/diculpa.mp4
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Disculpa/disculpa (1).mp4
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Disculpa/disculpa.mp4

Clase 'Inyeccion ': 3 videos
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Inyeccion /inyeccion (2).mp4
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Inyeccion /inyeccion (1).mp4
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Inyeccion /inyeccion.mp4

Clase 'Marron': 3 videos
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Marron/marron.mp4
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Marron/IMG_5869.mp4
  Procesando: /content/drive/MyDrive/SignProject/raw_videos/Marron/Marronf.mp4

Clase 'Por favor': 3 videos
  Procesando: /content/drive/MyDrive/Si

3. Carga de dataset secuencial + split por porcentaje

In [45]:
# 3. CARGA DE DATASET SECUENCIAL, NORMALIZACIÓN Y SPLIT ESTRATIFICADO

from sklearn.model_selection import train_test_split

data_path = os.path.join(PROCESSED_DIR, "landmarks_sequences_v2.npz")
data = np.load(data_path, allow_pickle=True)

# Claves del archivo .npz
X_seq = data["X_seq"].astype(np.float32)       # (n_samples, MAX_FRAMES, 225)
y_seq = data["y_seq"].astype(np.int64)         # (n_samples,)
label_names = data["label_names"].tolist()     # lista de nombres de clase

print("X_seq:", X_seq.shape)
print("y_seq:", y_seq.shape)
print("Clases:", label_names)

# 🔧 NORMALIZACIÓN GLOBAL (misma para train, val y luego app.py)
feature_mean = X_seq.mean(axis=(0, 1), keepdims=True).astype(np.float32)
feature_std = X_seq.std(axis=(0, 1), keepdims=True).astype(np.float32) + 1e-8

# Guardamos para usarlos luego en app.py / realtime
np.save(os.path.join(MODELS_DIR, "feature_mean_v2.npy"), feature_mean)
np.save(os.path.join(MODELS_DIR, "feature_std_v2.npy"), feature_std)
print("Guardados feature_mean_v2.npy y feature_std_v2.npy en MODELS_DIR")

# Aplicar normalización
X_seq = (X_seq - feature_mean) / feature_std

# 🔧 SPLIT ESTRATIFICADO CON SEMILLA FIJA
X_train_seq, X_val_seq, y_train_seq, y_val_seq = train_test_split(
    X_seq,
    y_seq,
    test_size=0.2,        # 20% validación
    stratify=y_seq,
    random_state=SEED,    # misma semilla que arriba
)

print("Train seq:", X_train_seq.shape, y_train_seq.shape)
print("Val seq:",   X_val_seq.shape,   y_val_seq.shape)


X_seq: (126, 20, 225)
y_seq: (126,)
Clases: ['Disculpa', 'Inyeccion ', 'Marron', 'Por favor', 'Resfriado', 'Tos', 'Verde']
Guardados feature_mean_v2.npy y feature_std_v2.npy en MODELS_DIR
Train seq: (100, 20, 225) (100,)
Val seq: (26, 20, 225) (26,)


4. Modelo LSTM (temporal)

In [46]:
# 4. MODELO LSTM TEMPORAL


num_classes = len(label_names)
max_frames = X_train_seq.shape[1]
n_features = X_train_seq.shape[2]

model_lstm = keras.Sequential([
    layers.Input(shape=(max_frames, n_features)),
    layers.Masking(mask_value=0.0),

    # LSTM más pequeño + regularización L2
    layers.LSTM(
        64,
        return_sequences=False,
        kernel_regularizer=keras.regularizers.l2(1e-4)
    ),

    layers.Dense(
        64,
        activation='relu',
        kernel_regularizer=keras.regularizers.l2(1e-4)
    ),
    layers.Dropout(0.4),

    layers.Dense(num_classes, activation='softmax')
])

model_lstm.compile(
    optimizer=keras.optimizers.Adam(5e-4),  # un poco más suave
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model_lstm.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)             │ (None, 20, 225)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        74,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,855 (308.03 KB)

 Trainable params: 78,855 (308.03 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# 4.1 ENTRENAMIENTO

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=10,
    min_lr=1e-5,
    verbose=1
)

history_lstm = model_lstm.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=300,
    batch_size=4,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

val_loss, val_acc = model_lstm.evaluate(X_val_seq, y_val_seq, verbose=0)
print("\nVAL ACCURACY LSTM:", val_acc)


Epoch 1/300
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 1.0000 - loss: 0.1731 - val_accuracy: 1.0000 - val_loss: 0.2787 - learning_rate: 5.0000e-04
Epoch 2/300
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 1.0000 - loss: 0.1859 - val_accuracy: 1.0000 - val_loss: 0.2466 - learning_rate: 5.0000e-04
Epoch 3/300
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 1.0000 - loss: 0.1803 - val_accuracy: 1.0000 - val_loss: 0.2341 - learning_rate: 5.0000e-04
Epoch 4/300
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 1.0000 - loss: 0.1801 - val_accuracy: 1.0000 - val_loss: 0.2142 - learning_rate: 5.0000e-04
Epoch 5/300
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 1.0000 - loss: 0.1323 - val_accuracy: 1.0000 - val_loss: 0.2053 - learning_rate: 5.0000e-04
Epoch 6/300
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.1204 - val_accuracy: 1.0000 - val_loss: 0.1856 - learning_rate: 5.0000e-04
Epoch 7/300
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 

In [49]:
keras_path_lstm = os.path.join(MODELS_DIR, "sign_model_lstm_v2.keras")
model_lstm.save(keras_path_lstm)
print("Modelo LSTM V2 guardado en:", keras_path_lstm)

labels_path = os.path.join(MODELS_DIR, "label_names_v2.json")
with open(labels_path, "w") as f:
    json.dump(label_names, f)
print("Listo. Guardado en:", labels_path)


Modelo LSTM V2 guardado en: /content/drive/MyDrive/SignProject/models/sign_model_lstm_v2.keras
Listo. Guardado en: /content/drive/MyDrive/SignProject/models/label_names_v2.json


In [50]:
loss_full, acc_full = model_lstm.evaluate(X_seq, y_seq, verbose=0)
print(f"Accuracy sobre TODO el dataset (solo referencia): {acc_full:.2f}")


Accuracy sobre TODO el dataset (solo referencia): 1.00


**6. Función para probar un video cualquiera (LSTM)**

In [51]:
# 6. USO DEL MODELO LSTM CON UN VIDEO


def load_lstm_model_and_labels():
    model_path = os.path.join(MODELS_DIR, "sign_model_lstm_v2.keras")
    labels_path = os.path.join(MODELS_DIR, "label_names_v2.json")
    mean_path = os.path.join(MODELS_DIR, "feature_mean.npy")
    std_path = os.path.join(MODELS_DIR, "feature_std.npy")

    model = keras.models.load_model(model_path)

    with open(labels_path, "r") as f:
        label_names = json.load(f)

    feature_mean = np.load(mean_path)
    feature_std = np.load(std_path)

    return model, label_names, feature_mean, feature_std



def preprocess_video_to_sequence(video_path, max_frames=MAX_FRAMES):
    """
    Procesa un video con MediaPipe Holistic y devuelve una secuencia
    (1, max_frames, 225) lista para el modelo LSTM.
    - Usamos pad_or_truncate para que sea consistente con el entrenamiento.
    - NO hay aleatoriedad aquí (solo center-crop si es muy largo).
    """
    cap = cv2.VideoCapture(video_path)

    frames_feats = []
    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        refine_face_landmarks=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    ) as holis:

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holis.process(frame_rgb)

            feats = extract_landmarks_from_results(results)  # (225,)
            frames_feats.append(feats)

    cap.release()

    if len(frames_feats) == 0:
        # Secuencia vacía, devolvemos todo ceros
        seq = np.zeros((max_frames, N_FEATURES), dtype=np.float32)
    else:
        seq_full = np.stack(frames_feats, axis=0)  # (T, 225)
        seq = pad_or_truncate(seq_full, max_frames=max_frames)

    # Añadimos dim de batch: (1, max_frames, 225)
    return seq[np.newaxis, ...].astype(np.float32)


def predict_video_lstm(video_path, max_frames=MAX_FRAMES):
    model, label_names, feature_mean, feature_std = load_lstm_model_and_labels()

    # Secuencia sin normalizar
    seq = preprocess_video_to_sequence(video_path, max_frames=max_frames)  # (1, T, 225)

    # Aplicamos la MISMA normalización que en entrenamiento
    seq_norm = (seq - feature_mean) / feature_std

    probs = model.predict(seq_norm, verbose=0)[0]  # (num_classes,)
    idx = int(np.argmax(probs))
    label = label_names[idx]
    conf = float(probs[idx])

    print(f"\nVideo: {video_path}")
    print(f"Predicción LSTM: {label} (confianza {conf:.2f})")
    print("\nDistribución de probabilidades:")
    for i, name in enumerate(label_names):
        print(f"  {name}: {probs[i]:.2f}")

    return label, conf, probs



In [52]:
test_video_path = os.path.join(BASE_DIR, "raw_videos", "Disculpa", "disculpa.mp4")
predict_video_lstm(test_video_path, max_frames=MAX_FRAMES)


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



Video: /content/drive/MyDrive/SignProject/raw_videos/Disculpa/disculpa.mp4
Predicción LSTM: Disculpa (confianza 1.00)

Distribución de probabilidades:
  Disculpa: 1.00
  Inyeccion : 0.00
  Marron: 0.00
  Por favor: 0.00
  Resfriado: 0.00
  Tos: 0.00
  Verde: 0.00


('Disculpa',
 0.9982185959815979,
 array([9.9821860e-01, 1.3293563e-04, 6.7201798e-04, 1.2750592e-04,
        2.1198047e-04, 4.0160911e-04, 2.3532695e-04], dtype=float32))

In [34]:
import os
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter

def evaluate_model_on_raw(RAW_DIR, max_frames=MAX_FRAMES):
    """
    Evalúa el modelo LSTM usando todos los videos RAW.
    Retorna:
      - reporte por clase
      - matriz de confusión
    """

    true_labels = []
    pred_labels = []

    classes = sorted([c for c in os.listdir(RAW_DIR) if os.path.isdir(os.path.join(RAW_DIR, c))])
    print("Clases encontradas:", classes)

    for cls in classes:
        class_dir = os.path.join(RAW_DIR, cls)
        video_files = [
            f for f in os.listdir(class_dir)
            if f.lower().endswith((".mp4", ".mov", ".avi", ".mkv"))
        ]

        print(f"\nEvaluando clase '{cls}': {len(video_files)} videos")

        for fname in video_files:
            video_path = os.path.join(class_dir, fname)
            label, conf, _ = predict_video_lstm(video_path, max_frames=max_frames)

            true_labels.append(cls)
            pred_labels.append(label)

            print(f"  {fname:<25} -> predicción: {label:<12} (conf {conf:.2f})")

    # ---------- MÉTRICAS ----------
    print("\n===== ACCURACY POR CLASE =====")
    counter_true = Counter(true_labels)
    counter_correct = Counter()

    for t, p in zip(true_labels, pred_labels):
        if t == p:
            counter_correct[t] += 1

    for cls in classes:
        total = counter_true[cls]
        correct = counter_correct[cls]
        acc = correct / total if total > 0 else 0
        print(f"{cls:<12}: {correct}/{total}  -> {acc*100:.1f}%")

    # Matriz de confusión
    print("\n===== MATRIZ DE CONFUSIÓN =====")
    cm = confusion_matrix(true_labels, pred_labels, labels=classes)
    print(cm)

    # Reporte sklearn bonito
    print("\n===== CLASSIFICATION REPORT =====")
    print(classification_report(true_labels, pred_labels, target_names=classes))

    return cm


# EJECUTAR
RAW_DIR = os.path.join(BASE_DIR, "raw_videos")
evaluate_model_on_raw(RAW_DIR, max_frames=MAX_FRAMES)


Clases encontradas: ['Disculpa', 'Inyeccion ', 'Marron', 'Por favor', 'Resfriado', 'Tos', 'Verde']

Evaluando clase 'Disculpa': 3 videos


KeyboardInterrupt: 